# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2022
topic: str = "QUÍMICA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. ¿Qué especie se determina cuando se valora ...
3,muestra que contiene Ca 2+ y Mg 2+ con EDTA a
4,pH=12 y murexida como indicador?:
5,
6,1. Ca2+.
7,2. Mg2+.
8,3. Ca2+ + Mg2+.
9,4. Ninguna de las dos especies.


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,1,43,4.0,85,4,127,4.0,169,2.0
1,2,4,44,2.0,86,1,128,3.0,170,3.0
2,3,2,45,2.0,87,1,129,4.0,171,1.0
3,4,1,46,4.0,88,2,130,2.0,172,4.0
4,5,4,47,NaN,89,1,131,4.0,173,2.0
5,6,3,48,4.0,90,3,132,4.0,174,3.0
6,7,4,49,4.0,91,1,133,2.0,175,2.0
7,8,2,50,2.0,92,2,134,4.0,176,2.0
8,9,3,51,3.0,93,3,135,4.0,177,3.0
9,10,4,52,3.0,94,3,136,3.0,178,4.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0    1. ¿Qué especie se determina cuando se valora una
1        muestra que contiene Ca 2+ y Mg 2+ con EDTA a
2                    pH=12 y murexida como indicador?:
3                                             1. Ca2+.
4                                             2. Mg2+.
5                                      3. Ca2+ + Mg2+.
6                      4. Ninguna de las dos especies.
7           2. Si valoramos 50,0 mL de HCl 0,050 M con
8         NaOH 0,100 M, ¿cuál es el valor del pH en el
9                              punto de equivalencia?:
Name: text, dtype: object

In [11]:
raw_exam_df.tail(10)

1844                            3. No se pueden corregir.
1845      4. Son debidos al azar y no se pueden eliminar,
1846                                 aunque sí minimizar.
1847    210. ¿Cuál es la enzima encargada de desenroll...
1848       doble hélice durante el proceso de replicación
1849                                            del DNA?:
1850                                          1. Primasa.
1851                                       2. DNA girasa.
1852                                     3. DNA helicasa.
1853                                   4. DNA polimerasa.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [12]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [13]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. ¿Qué especie se determina cuando se valora ...
1                                             1. Ca2+.
2                                             2. Mg2+.
3                                      3. Ca2+ + Mg2+.
4                      4. Ninguna de las dos especies.
5    2. Si valoramos 50,0 mL de HCl 0,050 M con NaO...
6                                                1. 1.
7                                                2. 2.
8                                                3. 5.
9                                                4. 7.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [14]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 10
Number of rows not ending with '.', ':' or '?' = 10
[140, 471, 473, 475, 477, 495, 511, 512, 692, 829]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [15]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [16]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [17]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [2, 86]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [18]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [19]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [20]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [21]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [22]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [23]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. ¿Qué especie se determina cuando se valora ...,1. Ca2+.,2. Mg2+.,3. Ca2+ + Mg2+.,4. Ninguna de las dos especies.
1,"2. Si valoramos 50,0 mL de HCl 0,050 M con NaO...",1. 1.,2. 2.,3. 5.,4. 7.
2,"3. En una disolución acuosa de KHA, ¿cuál es l...",1. [K+]= [HA-] + [OH-].,2. [K+]= [H2A] + [HA-] + [A2-].,3. [K+]= [HA-] + [OH-] + 2[A2-].,4. [K+]= [HA-] + [OH-] + 2[A2-].
3,4. ¿Qué proceso controla la corriente en la po...,1. La difusión.,2. La convección.,3. La adsorción.,4. La conducción.
4,5. ¿Cuál de las siguientes afirmaciones es FAL...,"1. Se aplica cuando, entre medidas sucesivas, ...",2. Se añaden a las muestras y a los patrones d...,"3. El patrón interno debe presentar, en el pro...",4. La concentración del analito de interés se ...
5,6. Una disolución 0.1 M de un ácido débil mono...,1. pKa = 2.,2. pKa = 5.,3. pKa = 7.,4. pKa = 9.
6,7. ¿Cuál de las siguientes afirmaciones es FAL...,1. Es un método que consiste en la valoración ...,2. El reactivo valorante del yodo generado es ...,3. El almidón (indicador) se añade cerca del p...,4. El punto final de la valoración yodométrica...
7,8. Las variaciones de presión en cromatografía...,1. No presentan ningún efecto sobre los tiempo...,2. Presentan un efecto muy marcado sobre el fa...,3. Tienen lugar durante l a separación debido ...,4. Empeoran significativamente los cromatogram...
8,9. La valoración de Karl Fischer está relacion...,1. La determinación de la dureza de un agua mi...,2. La determinación potenciométrica de haluros.,3. La determinación de agua.,4. La determinación del punto isoeléctrico de ...
9,10. Identifica la sustancia que se utiliza con...,1. Agua.,2. Metanol.,3. Hexano.,4. Dióxido de carbono.


In [24]:
exam_df_pivot.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4
200,201. ¿Cuál de las siguientes afirmaciones sobr...,1. Son representaciones gráficas del potencial...,2. Las especies más oxidadas se hallan en la p...,3. Las líneas oblicuas indican reacciones entr...,"4. Dos líneas oblicuas y paralelas, que repres..."
201,202. ¿Cuál de los siguientes compuestos experi...,1. 3-yodo-5-metilhexano.,2. 3-yodohexano.,3. 3-yodo-2-metilhexano.,4. 3-yodo-3-metilhexano.
202,203. ¿Qué mide la escala de Mohs?:,1. Acidez.,2. Conductividad.,3. Dureza.,4. Viscosidad.
203,204. La bilirrubina es:,1. Un biomarcador de función hepática.,2. Un biomarcador de función renal.,3. Un biomarcador de función cardiaca.,4. Un biomarcador de función cerebral.
204,205. ¿Qué tipo de electroforesis capilar emple...,1. Electroforesis capilar de zona (CZE).,2. Electroforesis capilar en gel (CGE).,3. Isotacoforesis capilar (CITP).,4. Isoelectroenfoque capilar (CIEF).
205,206. ¿A qué corresponde el código UFI que cons...,1. Código alfanumérico que permite identifica...,2. Código QR que incluye información sobre el ...,3. Código de barras que incluye información so...,4. Código QR que permite identificar de forma ...
206,207. La fluorescencia molecular:,1. Es más intensa al aumentar la temperatura.,2. Es más intensa al disminuir la viscosidad d...,3. Está particularmente favorecida en las molé...,4. Presenta un mayor rendimiento cuántico de e...
207,208. Considere la siguiente reacción: C (s) + ...,1. Se formará más H2O(g).,2. Se formarán más productos.,3. Es preciso conocer el valor de K p para sab...,"4. El valor de K c no cambia, por lo tanto, no..."
208,209. Toda medida lleva asociada un error. Los ...,1. Se corrigen realizando un número elevado de...,2. S e p r o d u c e n s i e m p r e e n e...,3. No se pueden corregir.,4. Son debidos al azar y no se pueden eliminar...
209,210. ¿Cuál es la enzima encargada de desenroll...,1. Primasa.,2. DNA girasa.,3. DNA helicasa.,4. DNA polimerasa.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,1.0
1,2,4.0
2,3,2.0
3,4,1.0
4,5,4.0
5,6,3.0
6,7,4.0
7,8,2.0
8,9,3.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. ¿Qué especie se determina cuando se valora ...,1. Ca2+.,2. Mg2+.,3. Ca2+ + Mg2+.,4. Ninguna de las dos especies.,1.0
1,"2. Si valoramos 50,0 mL de HCl 0,050 M con NaO...",1. 1.,2. 2.,3. 5.,4. 7.,4.0
2,"3. En una disolución acuosa de KHA, ¿cuál es l...",1. [K+]= [HA-] + [OH-].,2. [K+]= [H2A] + [HA-] + [A2-].,3. [K+]= [HA-] + [OH-] + 2[A2-].,4. [K+]= [HA-] + [OH-] + 2[A2-].,2.0
3,4. ¿Qué proceso controla la corriente en la po...,1. La difusión.,2. La convección.,3. La adsorción.,4. La conducción.,1.0
4,5. ¿Cuál de las siguientes afirmaciones es FAL...,"1. Se aplica cuando, entre medidas sucesivas, ...",2. Se añaden a las muestras y a los patrones d...,"3. El patrón interno debe presentar, en el pro...",4. La concentración del analito de interés se ...,4.0
5,6. Una disolución 0.1 M de un ácido débil mono...,1. pKa = 2.,2. pKa = 5.,3. pKa = 7.,4. pKa = 9.,3.0
6,7. ¿Cuál de las siguientes afirmaciones es FAL...,1. Es un método que consiste en la valoración ...,2. El reactivo valorante del yodo generado es ...,3. El almidón (indicador) se añade cerca del p...,4. El punto final de la valoración yodométrica...,4.0
7,8. Las variaciones de presión en cromatografía...,1. No presentan ningún efecto sobre los tiempo...,2. Presentan un efecto muy marcado sobre el fa...,3. Tienen lugar durante l a separación debido ...,4. Empeoran significativamente los cromatogram...,2.0
8,9. La valoración de Karl Fischer está relacion...,1. La determinación de la dureza de un agua mi...,2. La determinación potenciométrica de haluros.,3. La determinación de agua.,4. La determinación del punto isoeléctrico de ...,3.0
9,10. Identifica la sustancia que se utiliza con...,1. Agua.,2. Metanol.,3. Hexano.,4. Dióxido de carbono.,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. ¿Qué especie se determina cuando se valora ...,1. Ca2+.,2. Mg2+.,3. Ca2+ + Mg2+.,4. Ninguna de las dos especies.,1,2022
1,"2. Si valoramos 50,0 mL de HCl 0,050 M con NaO...",1. 1.,2. 2.,3. 5.,4. 7.,4,2022
2,"3. En una disolución acuosa de KHA, ¿cuál es l...",1. [K+]= [HA-] + [OH-].,2. [K+]= [H2A] + [HA-] + [A2-].,3. [K+]= [HA-] + [OH-] + 2[A2-].,4. [K+]= [HA-] + [OH-] + 2[A2-].,2,2022
3,4. ¿Qué proceso controla la corriente en la po...,1. La difusión.,2. La convección.,3. La adsorción.,4. La conducción.,1,2022
4,5. ¿Cuál de las siguientes afirmaciones es FAL...,"1. Se aplica cuando, entre medidas sucesivas, ...",2. Se añaden a las muestras y a los patrones d...,"3. El patrón interno debe presentar, en el pro...",4. La concentración del analito de interés se ...,4,2022
5,6. Una disolución 0.1 M de un ácido débil mono...,1. pKa = 2.,2. pKa = 5.,3. pKa = 7.,4. pKa = 9.,3,2022
6,7. ¿Cuál de las siguientes afirmaciones es FAL...,1. Es un método que consiste en la valoración ...,2. El reactivo valorante del yodo generado es ...,3. El almidón (indicador) se añade cerca del p...,4. El punto final de la valoración yodométrica...,4,2022
7,8. Las variaciones de presión en cromatografía...,1. No presentan ningún efecto sobre los tiempo...,2. Presentan un efecto muy marcado sobre el fa...,3. Tienen lugar durante l a separación debido ...,4. Empeoran significativamente los cromatogram...,2,2022
8,9. La valoración de Karl Fischer está relacion...,1. La determinación de la dureza de un agua mi...,2. La determinación potenciométrica de haluros.,3. La determinación de agua.,4. La determinación del punto isoeléctrico de ...,3,2022
9,10. Identifica la sustancia que se utiliza con...,1. Agua.,2. Metanol.,3. Hexano.,4. Dióxido de carbono.,4,2022


In [34]:
processed_exam_df.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
200,201. ¿Cuál de las siguientes afirmaciones sobr...,1. Son representaciones gráficas del potencial...,2. Las especies más oxidadas se hallan en la p...,3. Las líneas oblicuas indican reacciones entr...,"4. Dos líneas oblicuas y paralelas, que repres...",4,2022
201,202. ¿Cuál de los siguientes compuestos experi...,1. 3-yodo-5-metilhexano.,2. 3-yodohexano.,3. 3-yodo-2-metilhexano.,4. 3-yodo-3-metilhexano.,3,2022
202,203. ¿Qué mide la escala de Mohs?:,1. Acidez.,2. Conductividad.,3. Dureza.,4. Viscosidad.,3,2022
203,204. La bilirrubina es:,1. Un biomarcador de función hepática.,2. Un biomarcador de función renal.,3. Un biomarcador de función cardiaca.,4. Un biomarcador de función cerebral.,1,2022
204,205. ¿Qué tipo de electroforesis capilar emple...,1. Electroforesis capilar de zona (CZE).,2. Electroforesis capilar en gel (CGE).,3. Isotacoforesis capilar (CITP).,4. Isoelectroenfoque capilar (CIEF).,4,2022
205,206. ¿A qué corresponde el código UFI que cons...,1. Código alfanumérico que permite identifica...,2. Código QR que incluye información sobre el ...,3. Código de barras que incluye información so...,4. Código QR que permite identificar de forma ...,1,2022
206,207. La fluorescencia molecular:,1. Es más intensa al aumentar la temperatura.,2. Es más intensa al disminuir la viscosidad d...,3. Está particularmente favorecida en las molé...,4. Presenta un mayor rendimiento cuántico de e...,3,2022
207,208. Considere la siguiente reacción: C (s) + ...,1. Se formará más H2O(g).,2. Se formarán más productos.,3. Es preciso conocer el valor de K p para sab...,"4. El valor de K c no cambia, por lo tanto, no...",2,2022
208,209. Toda medida lleva asociada un error. Los ...,1. Se corrigen realizando un número elevado de...,2. S e p r o d u c e n s i e m p r e e n e...,3. No se pueden corregir.,4. Son debidos al azar y no se pueden eliminar...,2,2022
209,210. ¿Cuál es la enzima encargada de desenroll...,1. Primasa.,2. DNA girasa.,3. DNA helicasa.,4. DNA polimerasa.,3,2022


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [35]:
id_filter = [x for x in range(159, 200, 1)] + [203, 208, 209]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 44
Number of rows in filtered df = 44


**2. Showing the filtered DataFrame**

Using head and tail

In [36]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,160. ¿Cuál de las siguientes afirmaciones es c...,1. Son apolares.,2. Su función principal es la reserva energética.,"3. En el caso de que porten ácido siálico, se ...",4. Tienen dos residuos de ácidos grasos en su ...,3,2022
1,161. ¿Cuál de las siguientes afirmaciones es c...,1. La oxidación y la síntesis de ácidos grasos...,2. Tiene lugar en el citosol.,3. Tiene lugar con niveles plasmáticos bajos d...,4. Los ácidos grasos deben activarse (uniéndos...,4,2022
2,162. ¿Cuál de las siguientes afirmaciones es c...,1. El nitrógeno es liberado en el proceso de d...,2. El primer paso de la eliminación del nitróg...,3. Las aminotransferasas son inespecíficas y a...,4. Las reacciones de transaminación son altame...,2,2022
3,163. ¿Cuál de las siguientes afirmaciones es F...,1. La enzima clave es la ribonucleótido reduct...,2. La fuente de equivalentes reductores es el ...,3. El dATP inhibe alostéricamente la síntesis.,4. Se sintetizan directamente desde desoxirrib...,4,2022
4,164. ¿Cuál de las siguientes afirmaciones es c...,1. Se realiza únicamente en el inicio de la ru...,2. El glucagón inhibe la piruvato quinasa.,3. La insulina activa la síntesis de fructosa-...,4. La insulina inhibe la fosfofructoquinasa-1.,0,2022
5,165. Los valores de pKa del ácido aspártico so...,"1. 7,0.","2. 5,3.","3. 3,0.","4. 3,9.",3,2022
6,166. Las proteínas destinadas a ser secretadas...,1. Los polisomas libres.,2. El retículo endoplásmico rugoso.,3. El aparato de Golgi.,4. El retículo endoplásmico liso.,2,2022
7,167. El escualeno es un precursor de:,1. Esfingosina.,2. Prostaglandinas.,3. Ácido araquidónico.,4. Colesterol.,4,2022
8,168. Cada ciclo de oxidación de los ácidos gra...,1. Libera una molécula de acetil-CoA.,2. Consume dos moléculas de NADH.,3. Genera dos moléculas de FADH2.,4. Produce una deshidratación.,1,2022
9,169. La lanzadera malato-aspartato:,1. Permite la regeneración del NADH citosólico...,2. Sirve para pasar los electrones del NADH de...,3. Transfiere los electrones del NADH al citrato.,4. Es un complejo transportador de electrones.,2,2022


In [37]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
34,194. La alta fidelidad del p roceso de síntesi...,1. La actividad de la enzima peptidil transfer...,2. La presencia de factores de iniciación y de...,3. El acoplamiento entre el ribosoma y el RNA ...,4. La actividad correctora de la aminoacil-tRN...,4,2022
35,195. La impronta genética ( imprinting) es la:,1. Expresión diferencial de uno de los alelos.,2. Ausencia en el genoma de ciertos genes.,3. Expresión de genes causantes de enfermedades.,4. Presencia en el genoma de genes que permite...,1,2022
36,196. Las hormonas esteroideas actúan a nivel c...,1. Inhibiendo enzimas preexistentes.,2. Mediante la formación de un segundo mensajero.,3. Mediante la regulación de la expresión génica.,4. Modificando la concentr ación de calcio ión...,3,2022
37,"197. En referencia al fragmento Klenow, señale...",1. Es un fragmento de la enzima DNA polimerasa I.,2. Su gran ventaja es su estabilidad a altas t...,3. Se utiliza para el relleno de extremos 3´ d...,4. Tiene actividad de polimerasa y de 3´→5´ ex...,2,2022
38,198. ¿Qué es la secuencia Shine-Dalgarno?:,1. Una secuencia de RNA mensajero que solo exi...,2. Una secuencia situada a 6 o 7 nucleótidos d...,3. Una secuencia de DNA consenso en la región ...,4. Secuencia situada a 25 nucleótidos del siti...,1,2022
39,199. ¿Cómo afecta el aumen to de la temperatur...,1. Aumenta la velocidad y disminuye la afinida...,2. Disminuye la velocidad y aumenta la afinida...,3. Aumenta la velocidad y la afinidad de unión.,4. Disminuye la velocidad y la afinidad de unión.,1,2022
40,200. Los plásmidos son:,1. Fragmentos de DNA lineales con genes de res...,2. Moléculas de RNA de cadena doble y circular...,3. Moléculas de DNA circulares que se replican...,4. Moléculas de DNA circulares que se replican...,4,2022
41,204. La bilirrubina es:,1. Un biomarcador de función hepática.,2. Un biomarcador de función renal.,3. Un biomarcador de función cardiaca.,4. Un biomarcador de función cerebral.,1,2022
42,209. Toda medida lleva asociada un error. Los ...,1. Se corrigen realizando un número elevado de...,2. S e p r o d u c e n s i e m p r e e n e...,3. No se pueden corregir.,4. Son debidos al azar y no se pueden eliminar...,2,2022
43,210. ¿Cuál es la enzima encargada de desenroll...,1. Primasa.,2. DNA girasa.,3. DNA helicasa.,4. DNA polimerasa.,3,2022


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [38]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [39]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [40]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [41]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()